In [1]:
import copy
from few_shot import load_data_and_model, select_fs_examples, fewshots_finetuning


FREEZE_ZTSC = True
FREEZE_USP = False
LRS_USP = [0.1]
LRS_ZSTC = [0.0]
EPOCHS = [1]
SHOTS = [10]

# Load data and model.
tax_zero_shooter, train_data, valid_data = \
    load_data_and_model(n_train=1000, n_valid=100, freeze_zstc=FREEZE_ZTSC,
                        freeze_usp=FREEZE_USP, use_precomputed=FREEZE_ZTSC)
labels_to_consider = train_data.labels_levels[0]
for n_shots in SHOTS:
    examples_train, examples_valid = \
        select_fs_examples(n_shots, tax_zero_shooter, train_data, valid_data)
    for lr_zstc in LRS_ZSTC:
        for lr_usp in LRS_USP:
            for n_epochs in EPOCHS:
                model = copy.deepcopy(tax_zero_shooter)
                res, model = fewshots_finetuning(
                    model, labels_to_consider, examples_train,
                    examples_valid, n_shots=n_shots, lr_zstc=lr_zstc,
                    lr_usp=lr_usp, n_epochs=n_epochs, freeze_zstc=FREEZE_ZTSC,
                    freeze_usp=FREEZE_USP
                )
print(res)

C:\Users\LorenzoBongiovanni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[14:04:47][select_FS_examples.py]- INFO: Selecting examples for few-shot training
[14:04:54][few_shot.py]- INFO: 
------ Run Parameters -------
n_shots: 10, n_epochs: 1
lr_zstc: 0.0, lr_usp: 0.1
freeze_zstc: True, freeze_usp: False
100%|██████████| 10/10 [00:12<00:00,  1.25s/it]
[14:05:07][modeling.py]- INFO: Epoch 1/1 - Loss train: 2.986
[14:06:45][modeling.py]- INFO: SEEN: prec: 0.737, rec: 0.653, f1: 0.686
[14:06:45][modeling.py]- INFO: UNSEEN: prec: 0.500, rec: 0.467, f1: 0.483


{'p_seen': 0.7366946778711485, 'r_seen': 0.6530972995258709, 'f1_seen': 0.6860837438423646, 'p_unseen': 0.5, 'r_unseen': 0.4666666666666667, 'f1_unseen': 0.4827586206896552, 'n_shots': 10, 'lr_zstc': 0.0, 'lr_usp': 0.1, 'n_epochs': 1, 'freeze_zstc': True, 'freeze_usp': False}


In [2]:
model

TaxZeroShot()

In [23]:
labels_to_consider = set(valid_data.labels_levels[0])

In [10]:
seen = set([x.labels[0] for x in examples_train])
seen

{'Biochemistry',
 'Civil Engineering',
 'Computer Science',
 'Mechanical Engineering',
 'Medical Science',
 'Psychology'}

In [17]:
import numpy as np
id_seen = np.array([x.labels[0] in seen for x in examples_valid])
id_unseen = ~id_seen


False

In [18]:
res_flat, _ = model.forward([x.text for x in examples_valid])

In [24]:
res1 = [
    {k: v.item() for k, v in res.items() if k in labels_to_consider}
    for res in res_flat
]
# Get key with highest value.
res1

[{'Computer Science': 0.08944179117679596,
  'Medical Science': 0.2481178194284439,
  'Civil Engineering': 0.0,
  'Electrical Engineering': 0.06489451229572296,
  'Biochemistry': 0.31235626339912415,
  'Mechanical Engineering': 0.023395664989948273,
  'Psychology': 0.0},
 {'Computer Science': 0.06813511997461319,
  'Medical Science': 0.42890632152557373,
  'Civil Engineering': 0.02108026295900345,
  'Electrical Engineering': 0.04998236894607544,
  'Biochemistry': 0.11808229237794876,
  'Mechanical Engineering': 0.0,
  'Psychology': 0.44475024938583374},
 {'Computer Science': 0.044643811881542206,
  'Medical Science': 0.26245880126953125,
  'Civil Engineering': 0.0,
  'Electrical Engineering': 0.04851182550191879,
  'Biochemistry': 0.2913896441459656,
  'Mechanical Engineering': 0.0,
  'Psychology': 0.0},
 {'Computer Science': 0.04063086584210396,
  'Medical Science': 0.30119967460632324,
  'Civil Engineering': 0.0261042769998312,
  'Electrical Engineering': 0.07563834637403488,
  'Bioc

In [26]:
res2 = np.array([max(x, key=x.get) for x in res1])
res2

array(['Biochemistry', 'Psychology', 'Biochemistry', 'Psychology',
       'Biochemistry', 'Psychology', 'Medical Science',
       'Electrical Engineering', 'Computer Science',
       'Mechanical Engineering', 'Civil Engineering', 'Civil Engineering',
       'Medical Science', 'Medical Science', 'Electrical Engineering',
       'Electrical Engineering', 'Electrical Engineering', 'Psychology',
       'Electrical Engineering', 'Medical Science', 'Civil Engineering',
       'Medical Science', 'Electrical Engineering',
       'Electrical Engineering', 'Medical Science', 'Computer Science',
       'Electrical Engineering', 'Electrical Engineering',
       'Computer Science', 'Civil Engineering', 'Computer Science',
       'Psychology', 'Medical Science', 'Computer Science',
       'Biochemistry', 'Electrical Engineering', 'Psychology',
       'Electrical Engineering', 'Mechanical Engineering', 'Psychology',
       'Mechanical Engineering', 'Electrical Engineering',
       'Medical Science', 

In [28]:
res2[id_seen].shape

(85,)

In [32]:
from sklearn.metrics import precision_recall_fscore_support

labels_true = np.array([x.labels[0] for x in examples_valid])
targs = labels_true[id_unseen]
preds = res2[id_unseen]
precision_recall_fscore_support(targs, preds, average='macro')

(0.5, 0.4666666666666667, 0.4827586206896552, None)